# --- Day 23: Safe Cracking ---

This is one of the top floors of the nicest tower in EBHQ. The Easter Bunny's private office is here, complete with a safe hidden behind a painting, and who wouldn't hide a star in a safe behind a painting?

The safe has a digital screen and keypad for code entry. A sticky note attached to the safe has a password hint on it: "eggs". The painting is of a large rabbit coloring some eggs. You see 7.

When you go to type the code, though, nothing appears on the display; instead, the keypad comes apart in your hands, apparently having been smashed. Behind it is some kind of socket - one that matches a connector in your prototype computer! You pull apart the smashed keypad and extract the logic circuit, plug it into your computer, and plug your computer into the safe.

Now, you just need to figure out what output the keypad would have sent to the safe. You extract the assembunny code from the logic chip (your puzzle input).
The code looks like it uses almost the same architecture and instruction set that the monorail computer used! You should be able to use the same assembunny interpreter for this as you did there, but with one new instruction:

tgl x toggles the instruction x away (pointing at instructions like jnz does: positive means forward; negative means backward):

- For one-argument instructions, inc becomes dec, and all other one-argument instructions become inc.
- For two-argument instructions, jnz becomes cpy, and all other two-instructions become jnz.
- The arguments of a toggled instruction are not affected.
- If an attempt is made to toggle an instruction outside the program, nothing happens.
- If toggling produces an invalid instruction (like cpy 1 2) and an attempt is later made to execute that instruction, skip it instead.
- If tgl toggles itself (for example, if a is 0, tgl a would target itself and become inc a), the resulting instruction is not executed until the next time it is reached.

For example, given this program:

```
cpy 2 a
tgl a
tgl a
tgl a
cpy 1 a
dec a
dec a
```

- cpy 2 a initializes register a to 2.
- The first tgl a toggles an instruction a (2) away from it, which changes the third tgl a into inc a.
- The second tgl a also modifies an instruction 2 away from it, which changes the cpy 1 a into jnz 1 a.
- The fourth line, which is now inc a, increments a to 3.
- Finally, the fifth line, which is now jnz 1 a, jumps a (3) instructions ahead, skipping the dec a instructions.

In this example, the final value in register a is 3.

The rest of the electronics seem to place the keypad entry (the number of eggs, 7) in register a, run the code, and then send the value left in register a to the safe.

**What value should be sent to the safe?**

In [1]:
def get_input():
    with open(f'inputs/23.txt') as f:
        data = f.read().strip().split("\n")
    return data

data = get_input()
print(f"we have {len(data)} instructions")

we have 26 instructions


helper function to convert a string to int if possible.

In [2]:
def int_or_str(x):
    try:
        x = int(x)
    except:
        pass
    return x

int_or_str("a"), int_or_str("5")

('a', 5)

the main code:

In [3]:
from collections import defaultdict

registers = defaultdict(int)
registers["a"] = 7

i = 0
loop = 0

while i < len(data):
    ins = data[i].split()
    func = ins[0]
    x = int_or_str(ins[1])
    
    if len(ins) == 3:
        y = int_or_str(ins[2])
    
    if func == "tgl":
        x = registers[x] if type(x) is str else x
        if i + x >= len(data):
            pass
        else:
            new_ins = data[i+x].split()
            new_func = new_ins[0]
            new_x = int_or_str(new_ins[1])

            if len(new_ins) == 2:
                if new_func == "inc":
                    new_func = "dec"
                else:
                    new_func = "inc"
            else:
                new_y = int_or_str(new_ins[2])
                if new_func == "jnz":
                    new_func = "cpy"
                else:
                    new_func = "jnz"

            data[i+x] = new_func + data[i+x][3:]    
        
    elif func == "cpy":
        if type(y) is int: # this makes cpy invalid
            pass
        registers[y] = registers[x] if type(x) is str else x
    elif func == "inc":
        registers[x] += 1
    elif func == "dec":
        registers[x] -= 1
    elif func == "jnz":
        x = registers[x] if type(x) is str else x
        y = registers[y] if type(y) is str else y
        if x != 0:
            i += y-1
    i += 1
    loop += 1
    
    if loop % 100000 == 0:
        print(loop, i, registers)
    
registers

defaultdict(int, {'a': 12480, 'b': 1, 'd': 0, 'c': 0})

`12480` is the answer for part 1. some notes:

- damn this loop is now long and ugly. next time write a function for each instruction and then use a dispatch table like I did before.
- i only checked the cpy function if it is now invalid. Which was enough for part 1 but I'm sure this will fail for other inputs

# --- Part Two ---
The safe doesn't open, but it does make several angry noises to express its frustration.

You're quite sure your logic is working correctly, so the only other thing is... you check the painting again. As it turns out, colored eggs are still eggs. Now you count 12.

As you run the program with this new input, the prototype computer begins to overheat. You wonder what's taking so long, and whether the lack of any instruction more powerful than "add one" has anything to do with it. Don't bunnies usually multiply?

Anyway, **what value should actually be sent to the safe?**

So this looks like an optimization problem now, so taking a look at the instructions:

In [4]:
data

['cpy a b',
 'dec b',
 'cpy a d',
 'cpy 0 a',
 'cpy b c',
 'inc a',
 'dec c',
 'jnz c -2',
 'dec d',
 'jnz d -5',
 'dec b',
 'cpy b c',
 'cpy c d',
 'dec d',
 'inc c',
 'jnz d -2',
 'tgl c',
 'cpy -16 c',
 'cpy 1 c',
 'cpy 93 c',
 'cpy 80 d',
 'inc a',
 'dec d',
 'jnz d -2',
 'dec c',
 'jnz c -5']

this seems to be the multiplication loop, starting at `data[3]`:

1. `'cpy 0 a',` makes the register a `0` - only happens once

so the main mul loop:

1. `cpy b c` copies the num from register b into register c - making them equal
2.  `inc a` increases a by 1
3. `dec c` decreases c by 1
4. `jnz c -2` if c > 0 goes back 2
5. `dec d` decreases d by 1
6. `jnz d -5` runs this whole loop again while d > 0

than d is decreased, and the whole things runs again, calculating

so `register["a"] = b * d` ?

So lets try removing instructions data[4:9] and replacing them with `mul a b d` where mul replaces the value of register a with `b * d`:

In [5]:
data_2 = get_input()

dummy_ins = "jnz 0 0" # does nothing
mul_ins = "mul a b d" # the new mul ins

for i in range(4,10):
    if i == 4:
        data_2[i] = mul_ins
    else:
        data_2[i] = dummy_ins
data_2

['cpy a b',
 'dec b',
 'cpy a d',
 'cpy 0 a',
 'mul a b d',
 'jnz 0 0',
 'jnz 0 0',
 'jnz 0 0',
 'jnz 0 0',
 'jnz 0 0',
 'dec b',
 'cpy b c',
 'cpy c d',
 'dec d',
 'inc c',
 'jnz d -2',
 'tgl c',
 'cpy -16 c',
 'jnz 1 c',
 'cpy 93 c',
 'jnz 80 d',
 'inc a',
 'inc d',
 'jnz d -2',
 'inc c',
 'jnz c -5']

In [6]:
from collections import defaultdict

registers = defaultdict(int)
registers["a"] = 12

i = 0 # tracking the ins to execute

while i < len(data_2):
    ins = data_2[i].split()
    func = ins[0]
    x = int_or_str(ins[1])
    
    if len(ins) == 3:
        y = int_or_str(ins[2])
    
    if func == "tgl":
        x = registers[x] if type(x) is str else x
        
        if i + x >= len(data_2):
            pass
        else:
            new_ins = data_2[i+x].split()
            new_func = new_ins[0]
            new_x = int_or_str(new_ins[1])

            if len(new_ins) == 2:
                if new_func == "inc":
                    new_func = "dec"
                else:
                    new_func = "inc"
            else:
                new_y = int_or_str(new_ins[2])
                if new_func == "jnz":
                    new_func = "cpy"
                else:
                    new_func = "jnz"

            data_2[i+x] = new_func + data_2[i+x][3:]    
        
    elif func == "mul":
        registers[x] += registers["b"] * registers["d"]
        #registers["c"] = 0
        registers["d"] = 0
        
    elif func == "cpy":
        if type(y) is int: # this makes cpy invalid
            pass
        else:
            registers[y] = registers[x] if type(x) is str else x
    elif func == "inc":
        registers[x] += 1
    elif func == "dec":
        registers[x] -= 1
    elif func == "jnz":
        x = registers[x] if type(x) is str else x
        y = registers[y] if type(y) is str else y
        
        if x != 0:
            i += y-1
    i += 1
    
registers

defaultdict(int, {'a': 479009040, 'b': 1, 'd': 0, 'c': 0})

`479009040` is the answer for part 2. Now there is another multiplication loop at the end which I could replace with the `mul` function but its fast enough already.

## Notes:

- i got stuck at a very stupid problem - where the first run of my while loop modified the original data instructions, so even when I had the right code I was getting the wrong answer. I had to go back to read in the inputs again, modify, and feed to the while loop. Its a good reminder to use functions and keep track of changes made to the inputs.
- Optimization is tricky. drawing is helpful.